In [1]:
%pip install python-constraint

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from constraint import Problem, AllDifferentConstraint  # the constraint package

## Input Format

In [3]:
# the list of teams
teams = [ "A", "B", "C", "D" ]
# matrix that lists available locations to organize the matches on
location_available = {
    "A": [ 1,       4,    6, 7, 8,    10 ], # location of team A is available for the listed time slots
    "B": [    2,    4, 5, 6, 7,    9,    ],
    "C": [    2, 3,    5,       8,    10 ],
    "D": [ 1,    3,       6,    8, 9, 10 ],
}
# NOTE: this is just an example problem, your implementation should work for any scheduling problem with this structure

In [ ]:
problem = Problem()

# iterate through all possible team tuples
for h in teams:                                 
    for g in teams:                      
        #check duplicate tuples   
        if h != g:
            # Add the tuple using the location of home stadium h
            problem.addVariable((h,g), location_available[h])

## Constraints

Model the constraints speficied in the exercise here. Add a section for each constraint.

In [5]:
# TODO model a constraint here
#############################Constraint 1#############################
season_split = 5                    #define 5 to test with the given example, later 17 for actual Bundesliga
def check_half_season(day1, day2):
    # check for gamedays to be in different season halfs
    if (day1 <= season_split and day2 > season_split) or (day1 > season_split and day2 <= season_split): 
        return True
    else:
        return False

for h in teams:
    for g in teams:
        # collect all possible mathes
        if h < g:
            problem.addConstraint(check_half_season, [(h,g),(g,h)])

#############################Constraint 2 & 3#############################
# Note: Constraint 3 (at most one match per location) is implicitly satisfied here, 
# because each location is exclusively tied to one home team which plays at most once per slot.

for h in teams:
    # collect all match variables involving team h
    matches = []
    for g in teams:
        if h != g:
            matches.append((h,g))
            matches.append((g,h))
            # all the matches of a single team should be on different days
    problem.addConstraint(AllDifferentConstraint(), matches)

#############################Constraint 4#############################
# Constraint 4: Venue Validity
# Requirement: "Every pairing {ti, tj} should be played once in ti and once in tj."
# Status: IMPLICITLY SATISFIED
# Explanation:
# This is handled during the variable creation phase. 
# - We generated a variable (h, g) and assigned it the domain of 'h' (location_available[h]).
# - We generated a variable (g, h) and assigned it the domain of 'g' (location_available[g]).
# Therefore, it is structurally guaranteed that one match takes place at h's home 
# and the other at g's home.

#############################Constraint 5#############################
# Constraint 5: Sequential Season Halves
# Requirement: "The first half must be completed before the second half starts."
# Status: IMPLICITLY SATISFIED
# Explanation:
# This is covered by the logic of Constraint 1 (Season Split).
# We enforced a strict numerical boundary using 'season_split' (e.g., 5 or 17).
# - Any match assigned to the first half is constrained to be <= season_split.
# - Any match assigned to the second half is constrained to be > season_split.
# This mathematically necessitates that the entire first half finishes before 
# any match of the second half begins.


#############################Constraint 6#############################
# Mirror Logic: Enforce relative ordering consistency between season halves.
# If Pair A plays before Pair B in the first half, they must also play in that order in the second half.

def mirror_logic(home1, away1, home2, away2):
    # Determine chronologically which match is the "First Leg" (Hinrunde) 
    # and which is the "Second Leg" (Rückrunde) for both pairings.
    # Note: We use min() because our variables define location (Home/Away), not time.
    # The earlier date (min) is always the first season half.
    leg1_pair1 = min(home1, away1) # First leg of Pair 1
    leg2_pair1 = max(home1, away1) # Second leg of Pair 1
    
    leg1_pair2 = min(home2, away2) # First leg of Pair 2
    leg2_pair2 = max(home2, away2) # Second leg of Pair 2

    # Check for consistency in ordering:
    # Case 1: Pair 1 played before Pair 2 in the first half -> must be same in second half.
    if leg1_pair1 <= leg1_pair2 and leg2_pair1 <= leg2_pair2:
        return True
    
    # Case 2: Pair 2 played before Pair 1 in the first half -> must be same in second half.
    if leg1_pair1 >= leg1_pair2 and leg2_pair1 >= leg2_pair2:
        return True
        
    return False

pairings = []
# Collect all unique base matchups (ignoring home/away order for now)
for h in teams:
    for g in teams:
        if h < g: # Ensure we only collect {A, B} and not {B, A} to avoid duplicates
            pairings.append((h,g))

# Compare every pairing against every other pairing
for i in range(len(pairings)):
    for j in range(i+1, len(pairings)):
        pair1 = pairings[i]             
        pair2 = pairings[j]
        
        # pass all 4 relevant variables to the constraint:
        # 1. Pair 1 (Home) & Pair 1 (Away)
        # 2. Pair 2 (Home) & Pair 2 (Away)
        # reconstruct the 'Away' variable simply by swapping.
        pair1_away = (pair1[1], pair1[0])
        pair2_away = (pair2[1], pair2[0])
        
        problem.addConstraint(mirror_logic, [pair1, pair1_away, pair2, pair2_away])

## Solution

In [ ]:
def print_solution(sol, filename = "scheduling.log"):
    with open(filename, "w") as f:
        # sort the solution with the gamedays
        sorted_schedule = sorted(sol.items(), key = lambda x:x[1])
        f.write("-------- Bundesliga scheduling solution --------\n")
        # collect and format
        for match, day in sorted_schedule:
            home_team = match[0]
            guest_team = match[1]
            line = f"Day {day}: {home_team} vs {guest_team} (at {home_team})"
            f.write(line + "\n")
        f.write("------------------------------------\n")
        print(f"\nSuccessfully saved schedule to {filename}")        

In [7]:
solutions = problem.getSolutions()

In [8]:
print(f"Number of solutions: {len(solutions)}\n")
print_solution(solutions[0])

Number of solutions: 576


Successfully saved schedule to scheduling.log
